In [1]:
import streamlit as st
import pandas as pd

In [2]:
from Features import world_constructions as WCf

In [3]:
from Features import search_from_luxury_brands as Sf

### Loading data

In [4]:
df = WCf.world_construction.init_luxury_gallery()

In [5]:
vectorestore = WCf.world_construction.init_chroma_db()

c:\Users\juand\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\juand\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\juand\anaconda3\Lib\site-packages\transformers\utils\generic.py:482: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\

In [6]:
description = df.iloc[0].Summary

In [7]:
description

'The item is a Gucci silk shirt featuring an optical GG monogram print. It is designed with a regular fit and features long sleeves with buttoned cuffs. The neckline is a classic pointed collar, adding a touch of formality. The material used is high-quality silk, which gives the shirt a luxurious feel and fluid drape, suitable for all seasons due to its breathability and lightweight nature. The primary colors used are a deep lavender (5467) and white, which are intricately interwoven to create a striking visual effect. The pattern placement is consistent throughout the shirt, ensuring a cohesive look. The texture is smooth and slightly glossy, typical of silk fabric, with no transparency, making it appropriate for both formal and semi-formal occasions. There are no pockets, embellishments, or lapels, which maintains a sleek and streamlined silhouette. The shirt’s length is standard, typically falling just below the waistline, and it is finished with concealed buttons for a minimalist a

### Feature 1: Find similar pieces from luxury brands in retail brands 

In [8]:
entity = Sf.seacrh_from_luxury_brands(description=description, vectorstore=vectorestore)

In [9]:
answer = entity.search_similarity_from_description()

In [10]:
answer['answer']

"Based on the description provided, here are the best options that closely match the elegant silk shirt with an optical monogram print:\n\n1. **Option 1:** The item is a Gucci silk shirt featuring an optical GG monogram print. It is designed with a regular fit and features long sleeves with buttoned cuffs. The neckline is a classic pointed collar, adding a touch of formality. The material used is high-quality silk, which gives the shirt a luxurious feel and fluid drape, suitable for all seasons due to its breathability and lightweight nature. The primary colors used are a deep lavender (5467) and white, which are intricately interwoven to create a striking visual effect. The pattern placement is consistent throughout the shirt, ensuring a cohesive look. The texture is smooth and slightly glossy, typical of silk fabric, with no transparency, making it appropriate for both formal and semi-formal occasions. There are no pockets, embellishments, or lapels, which maintains a sleek and strea

### Feature 2: Find pieces from user prompt description ( Text prompt to suggestions)

### Feature 3: Find pieces similar to user provided piece ( Image to suggestions)